### Bibliotecas

In [1]:
import time
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import pandas as pd
import re

### Imóveis comum

In [2]:
# URL inicial
start_url = "https://www.zapimoveis.com.br/venda/apartamentos/pr+cascavel/?pagina=1"

In [3]:
def accept_cookies(driver):
    """Aceita cookies caso o banner apareça."""
    try:
        cookies_button = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Aceitar todos') or contains(text(), 'Aceitar')]"))
        )
        cookies_button.click()
        print("Cookies aceitos.")
    except TimeoutException:
        print("Banner de cookies não encontrado ou já aceito.")

In [4]:
def scroll_a_little(driver):
    """Rola a página levemente para carregar novos elementos."""
    print("Rolando um pouco a página...")
    driver.execute_script("window.scrollBy(0, 300);")  # Rolagem menor
    time.sleep(0.5)  # Tempo menor de espera

In [5]:
def extract_id_from_url(url):
    """
    Extrai o ID do imóvel do URL.
    """
    # Remove a barra final, se existir, e pega o texto após o último hífen
    return url.rstrip('/').split('-')[-1]

In [33]:
def scrape_property_data(driver, property_id):
    """Coleta os dados de um imóvel na aba atual."""
    data = {"ID": property_id}  # Adicionar ID como chave primária
    try:
        features = [feature.text for feature in driver.find_elements(By.CSS_SELECTOR, "span[data-cy='ldp-propertyFeatures-txt']")]
        data['Área'] = features[0] if len(features) > 0 else "Não encontrado"
        data['Quartos'] = features[1] if len(features) > 1 else "Não encontrado"
        data['Banheiros'] = features[2] if len(features) > 2 else "Não encontrado"
        data['Suítes'] = features[3] if len(features) > 3 else "Não encontrado"
    except Exception:
        data['Área'] = data['Quartos'] = data['Banheiros'] = data['Suítes'] = "Não encontrado"

    try:
        meta_description = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "meta[name='description']"))
        )
        description_content = meta_description.get_attribute("content")
        price_match = re.search(r"R\$ [\d.,]+", description_content)
        data['Preço'] = price_match.group(0) if price_match else "Não encontrado"
    except TimeoutException:
        data['Preço'] = "Não encontrado"

    try:
        address_element = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "a.listing-breadcrumb__dark-link"))
        )
        data['Endereço'] = address_element.text
    except TimeoutException:
        data['Endereço'] = "Não encontrado"

    return data

In [6]:
def click_next_page(driver):
    """Clica no botão de próxima página, se disponível."""
    try:
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "button[data-testid='next-page']"))
        )
        next_button.click()
        time.sleep(2)
        print("Avançando para a próxima página...")
        return True
    except TimeoutException:
        print("Botão de próxima página não encontrado. Fim da navegação.")
        return False

In [7]:
def save_progress(processed_ids):
    """Salva os IDs processados para retomada em caso de falha."""
    with open("progress.txt", "w") as file:
        file.write("\n".join(processed_ids))

In [8]:
def load_progress():
    """Carrega os IDs processados de um arquivo, se existir."""
    try:
        with open("progress.txt", "r") as file:
            return set(file.read().splitlines())
    except FileNotFoundError:
        return set()

In [34]:
def scrape_zapimoveis():
    options = uc.ChromeOptions()
    options.headless = False
    driver = uc.Chrome(options=options)
    columns = ["ID", "Área", "Quartos", "Banheiros", "Suítes", "Preço", "Endereço"]
    df = pd.DataFrame(columns=columns)
    processed_ids = load_progress()  # Carrega IDs processados de arquivo

    try:
        driver.get(start_url)
        accept_cookies(driver)

        listings = driver.find_elements(By.CSS_SELECTOR, "h2[data-cy='rp-cardProperty-location-txt']")
        while listings:
            for i in range(len(listings)):
                try:
                    listings = driver.find_elements(By.CSS_SELECTOR, "h2[data-cy='rp-cardProperty-location-txt']")
                    link = listings[i].find_element(By.XPATH, "./ancestor::a").get_attribute("href")
                    property_id = extract_id_from_url(link)

                    if property_id in processed_ids:
                        print(f"Imóvel já processado: ID {property_id}")
                        continue

                    print(f"\nProcessando imóvel {i + 1} - ID: {property_id}")
                    processed_ids.add(property_id)  # Adiciona ID ao conjunto de processados
                    save_progress(processed_ids)  # Salva progresso continuamente

                    start_time = time.time()
                    WebDriverWait(driver, 5).until(EC.element_to_be_clickable(listings[i])).click()
                    WebDriverWait(driver, 10).until(EC.number_of_windows_to_be(2))
                    driver.switch_to.window(driver.window_handles[1])
                    accept_cookies(driver)

                    data = scrape_property_data(driver, property_id)
                    df = pd.concat([df, pd.DataFrame([data])], ignore_index=True)

                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])
                    elapsed_time = time.time() - start_time
                    print(f"Tempo entre clique e retorno: {elapsed_time:.2f} segundos")

                    scroll_a_little(driver)
                except Exception as e:
                    print(f"Erro ao processar imóvel: {e}")

            # Tenta avançar para a próxima página
            if not click_next_page(driver):
                break  # Sai do loop caso não haja mais páginas

            # Atualiza a lista de imóveis para a nova página
            listings = driver.find_elements(By.CSS_SELECTOR, "h2[data-cy='rp-cardProperty-location-txt']")

    finally:
        driver.quit()
        df.to_csv("imoveis_zapimoveis.csv", index=False, encoding="utf-8")
        print("Dados salvos no arquivo 'imoveis_zapimoveis.csv'.")

if __name__ == '__main__':
    scrape_zapimoveis()

Cookies aceitos.
Imóvel já processado: ID 2734956032
Imóvel já processado: ID 2577416376
Imóvel já processado: ID 2759307214
Imóvel já processado: ID 2634945881
Imóvel já processado: ID 2760815615
Imóvel já processado: ID 2760816022
Imóvel já processado: ID 2760814429
Imóvel já processado: ID 2760816145
Imóvel já processado: ID 2762689441
Imóvel já processado: ID 2760815719
Imóvel já processado: ID 2760815634
Imóvel já processado: ID 2760815623
Imóvel já processado: ID 2762689551
Imóvel já processado: ID 2760815551
Imóvel já processado: ID 2762469796
Avançando para a próxima página...
Imóvel já processado: ID 2734956032
Imóvel já processado: ID 2577416376
Imóvel já processado: ID 2759307214
Imóvel já processado: ID 2634945881
Imóvel já processado: ID 2760815615
Imóvel já processado: ID 2760816022
Imóvel já processado: ID 2760814429
Imóvel já processado: ID 2760816145
Imóvel já processado: ID 2762689441
Imóvel já processado: ID 2760815719
Imóvel já processado: ID 2760815634
Imóvel já pr

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=131.0.6778.109)
Stacktrace:
	GetHandleVerifier [0x00CF3433+25059]
	(No symbol) [0x00C7CE34]
	(No symbol) [0x00B5BEC3]
	(No symbol) [0x00B3D93B]
	(No symbol) [0x00BC800F]
	(No symbol) [0x00BDAE49]
	(No symbol) [0x00BC1C96]
	(No symbol) [0x00B93FAC]
	(No symbol) [0x00B94F3D]
	GetHandleVerifier [0x00FE5593+3113795]
	GetHandleVerifier [0x00FFA25A+3198986]
	GetHandleVerifier [0x00FF2A32+3168226]
	GetHandleVerifier [0x00D932A0+680016]
	(No symbol) [0x00C8577D]
	(No symbol) [0x00C82A28]
	(No symbol) [0x00C82BC5]
	(No symbol) [0x00C75820]
	BaseThreadInitThunk [0x75F77BA9+25]
	RtlInitializeExceptionChain [0x773CC0CB+107]
	RtlClearBits [0x773CC04F+191]


### novos dados para a coleta OK

In [ ]:
def scrape_property_data(driver, property_id):
    """Coleta os dados de um imóvel na aba atual."""
    data = {"ID": property_id}  # Adicionar ID como chave primária

    # Coletar características principais
    try:
        features = [feature.text for feature in driver.find_elements(By.CSS_SELECTOR, "span[data-cy='ldp-propertyFeatures-txt']")]
        data['Área'] = features[0] if len(features) > 0 else "Não encontrado"
        data['Quartos'] = features[1] if len(features) > 1 else "Não encontrado"
        data['Banheiros'] = features[2] if len(features) > 2 else "Não encontrado"
        data['Vagas de Carro'] = features[3] if len(features) > 3 else "Não encontrado"
    except Exception:
        data['Área'] = data['Quartos'] = data['Banheiros'] = data['Vagas de Carro'] = "Não encontrado"

    # Coletar preço
    try:
        meta_description = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "meta[name='description']"))
        )
        description_content = meta_description.get_attribute("content")
        price_match = re.search(r"R\$ [\d.,]+", description_content)
        data['Preço'] = price_match.group(0) if price_match else "Não encontrado"
    except TimeoutException:
        data['Preço'] = "Não encontrado"

    # Coletar endereço
    try:
        address_element = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "p.address-info-value"))
        )
        data['Endereço'] = address_element.text
    except TimeoutException:
        data['Endereço'] = "Não encontrado"

    # Coletar itens adicionais
    try:
        additional_items = [item.text for item in driver.find_elements(By.CSS_SELECTOR, "span.amenities-item-text[data-cy='ldp-propertyFeatures-txt']")]
        data['Itens Adicionais'] = ", ".join(additional_items) if additional_items else "Não encontrado"
    except Exception:
        data['Itens Adicionais'] = "Não encontrado"

    return data

In [31]:
def scrape_zapimoveis():
    options = uc.ChromeOptions()
    options.headless = False
    driver = uc.Chrome(options=options)
    columns = ["ID", "Área", "Quartos", "Banheiros", "Vagas de Carro", "Preço", "Endereço", "Itens Adicionais"]
    df = pd.DataFrame(columns=columns)
    processed_ids = load_progress()  # Carrega IDs processados de arquivo

    try:
        driver.get(start_url)
        accept_cookies(driver)

        listings = driver.find_elements(By.CSS_SELECTOR, "h2[data-cy='rp-cardProperty-location-txt']")
        while listings:
            for i in range(len(listings)):
                try:
                    listings = driver.find_elements(By.CSS_SELECTOR, "h2[data-cy='rp-cardProperty-location-txt']")
                    link = listings[i].find_element(By.XPATH, "./ancestor::a").get_attribute("href")
                    property_id = extract_id_from_url(link)

                    if property_id in processed_ids:
                        print(f"Imóvel já processado: ID {property_id}")
                        continue

                    print(f"\nProcessando imóvel {i + 1} - ID: {property_id}")
                    processed_ids.add(property_id)  # Adiciona ID ao conjunto de processados
                    save_progress(processed_ids)  # Salva progresso continuamente

                    start_time = time.time()
                    WebDriverWait(driver, 5).until(EC.element_to_be_clickable(listings[i])).click()
                    WebDriverWait(driver, 10).until(EC.number_of_windows_to_be(2))
                    driver.switch_to.window(driver.window_handles[1])
                    accept_cookies(driver)

                    data = scrape_property_data(driver, property_id)
                    df = pd.concat([df, pd.DataFrame([data])], ignore_index=True)

                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])
                    elapsed_time = time.time() - start_time
                    print(f"Tempo entre clique e retorno: {elapsed_time:.2f} segundos")

                    scroll_a_little(driver)
                except Exception as e:
                    print(f"Erro ao processar imóvel: {e}")

            # Tenta avançar para a próxima página
            if not click_next_page(driver):
                break  # Sai do loop caso não haja mais páginas

            # Atualiza a lista de imóveis para a nova página
            listings = driver.find_elements(By.CSS_SELECTOR, "h2[data-cy='rp-cardProperty-location-txt']")

    finally:
        driver.quit()
        df.to_csv("imoveis_zapimoveis.csv", index=False, encoding="utf-8")
        print("Dados salvos no arquivo 'imoveis_zapimoveis.csv'.")

if __name__ == '__main__':
    scrape_zapimoveis()

Cookies aceitos.
Imóvel já processado: ID 2759307214
Imóvel já processado: ID 2637525490
Imóvel já processado: ID 2589922973
Imóvel já processado: ID 2634945881
Imóvel já processado: ID 2760815615
Imóvel já processado: ID 2760816022
Imóvel já processado: ID 2760814429
Imóvel já processado: ID 2760816145
Imóvel já processado: ID 2762689441
Imóvel já processado: ID 2760815719
Imóvel já processado: ID 2760815634
Imóvel já processado: ID 2760815623
Imóvel já processado: ID 2762689551
Imóvel já processado: ID 2760815551
Imóvel já processado: ID 2762469796
Avançando para a próxima página...
Imóvel já processado: ID 2759307214
Imóvel já processado: ID 2637525490
Imóvel já processado: ID 2589922973
Imóvel já processado: ID 2634945881
Imóvel já processado: ID 2760815615
Imóvel já processado: ID 2760816022
Imóvel já processado: ID 2760814429
Imóvel já processado: ID 2760816145
Imóvel já processado: ID 2762689441
Imóvel já processado: ID 2760815719
Imóvel já processado: ID 2760815634
Imóvel já pr

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=131.0.6778.109)
Stacktrace:
	GetHandleVerifier [0x00293433+25059]
	(No symbol) [0x0021CE34]
	(No symbol) [0x000FBEC3]
	(No symbol) [0x000DD93B]
	(No symbol) [0x0016800F]
	(No symbol) [0x0017AE49]
	(No symbol) [0x00161C96]
	(No symbol) [0x00133FAC]
	(No symbol) [0x00134F3D]
	GetHandleVerifier [0x00585593+3113795]
	GetHandleVerifier [0x0059A25A+3198986]
	GetHandleVerifier [0x00592A32+3168226]
	GetHandleVerifier [0x003332A0+680016]
	(No symbol) [0x0022577D]
	(No symbol) [0x00222A28]
	(No symbol) [0x00222BC5]
	(No symbol) [0x00215820]
	BaseThreadInitThunk [0x75F77BA9+25]
	RtlInitializeExceptionChain [0x773CC0CB+107]
	RtlClearBits [0x773CC04F+191]
	(No symbol) [0x00000000]


### Ajustando coleta de novos dados

In [11]:
def scrape_property_data(driver, property_id, ad_text):
    """Coleta os dados de um imóvel na aba atual."""
    data = {"ID": property_id}  # Adicionar ID como chave primária

    # Coletar características principais
    try:
        features = [feature.text for feature in driver.find_elements(By.CSS_SELECTOR, "span[data-cy='ldp-propertyFeatures-txt']")]
        data['Área'] = features[0] if len(features) > 0 else "Não encontrado"
        data['Quartos'] = features[1] if len(features) > 1 else "Não encontrado"
        data['Banheiros'] = features[2] if len(features) > 2 else "Não encontrado"
        data['Vagas de Carro'] = features[3] if len(features) > 3 else "Não encontrado"
    except Exception:
        data['Área'] = data['Quartos'] = data['Banheiros'] = data['Vagas de Carro'] = "Não encontrado"

    # Coletar preço
    try:
        meta_description = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "meta[name='description']"))
        )
        description_content = meta_description.get_attribute("content")
        price_match = re.search(r"R\$ [\d.,]+", description_content)
        data['Preço'] = price_match.group(0) if price_match else "Não encontrado"
    except TimeoutException:
        data['Preço'] = "Não encontrado"

    # Coletar endereço
    try:
        address_element = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "p.address-info-value"))
        )
        data['Endereço'] = address_element.text
    except TimeoutException:
        data['Endereço'] = "Não encontrado"

    # Coletar itens adicionais
    try:
        additional_items = [item.text for item in driver.find_elements(By.CSS_SELECTOR, "span.amenities-item-text[data-cy='ldp-propertyFeatures-txt']")]
        data['Itens Adicionais'] = ", ".join(additional_items) if additional_items else "Não encontrado"
    except Exception:
        data['Itens Adicionais'] = "Não encontrado"

    # Usar o texto do anúncio coletado previamente
    data['Texto do Anúncio'] = ad_text

    return data

#### Função scrape_zapimoveis mais eficiente capturando o texto do ímovel sem se misturar com o ID. Porém coletando o mesmo texto para todos os imóveis.

In [12]:
def scrape_zapimoveis():
    options = uc.ChromeOptions()
    options.headless = False
    driver = uc.Chrome(options=options)
    columns = ["ID", "Área", "Quartos", "Banheiros", "Vagas de Carro", "Preço", "Endereço", "Itens Adicionais", "Texto do Anúncio"]
    df = pd.DataFrame(columns=columns)
    processed_ids = load_progress()  # Carrega IDs processados de arquivo

    try:
        driver.get(start_url)
        accept_cookies(driver)

        listings = driver.find_elements(By.CSS_SELECTOR, "h2[data-cy='rp-cardProperty-location-txt']")
        while listings:
            for i in range(len(listings)):
                try:
                    listings = driver.find_elements(By.CSS_SELECTOR, "h2[data-cy='rp-cardProperty-location-txt']")
                    link = listings[i].find_element(By.XPATH, "./ancestor::a").get_attribute("href")
                    property_id = extract_id_from_url(link)

                    if property_id in processed_ids:
                        print(f"Imóvel já processado: ID {property_id}")
                        continue

                    # Coletar texto do anúncio usando o seletor fornecido
                    description_selector = "#__next > main > section > div > form > div.result-wrapper__content > div.result-wrapper__children > div.listing-wrapper > div > div:nth-child(2) > div > a > div > div.BaseCard_card__Ci4Ny.BaseCard_card--horizontal__GgDlY > div.BaseCard_card__content__pL2Vc.w-full.p-3 > div:nth-child(2) > p"
                    description_element = driver.find_element(By.CSS_SELECTOR, description_selector)
                    ad_text = description_element.text if description_element else "Não encontrado"

                    print(f"\nProcessando imóvel {i + 1} - ID: {property_id}")
                    print(f"Texto do Anúncio: {ad_text[:100]}...")  # Mostra uma parte do texto para depuração

                    processed_ids.add(property_id)  # Adiciona ID ao conjunto de processados
                    save_progress(processed_ids)  # Salva progresso continuamente

                    start_time = time.time()
                    WebDriverWait(driver, 5).until(EC.element_to_be_clickable(listings[i])).click()
                    WebDriverWait(driver, 10).until(EC.number_of_windows_to_be(2))
                    driver.switch_to.window(driver.window_handles[1])
                    accept_cookies(driver)

                    data = scrape_property_data(driver, property_id, ad_text)
                    df = pd.concat([df, pd.DataFrame([data])], ignore_index=True)

                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])
                    elapsed_time = time.time() - start_time
                    print(f"Tempo entre clique e retorno: {elapsed_time:.2f} segundos")

                    scroll_a_little(driver)
                except Exception as e:
                    print(f"Erro ao processar imóvel: {e}")

            # Tenta avançar para a próxima página
            if not click_next_page(driver):
                break  # Sai do loop caso não haja mais páginas

            # Atualiza a lista de imóveis para a nova página
            listings = driver.find_elements(By.CSS_SELECTOR, "h2[data-cy='rp-cardProperty-location-txt']")

    finally:
        driver.quit()
        df.to_csv("imoveis_zapimoveis.csv", index=False, encoding="utf-8")
        print("Dados salvos no arquivo 'imoveis_zapimoveis.csv'.")

if __name__ == '__main__':
    scrape_zapimoveis()

Cookies aceitos.

Processando imóvel 1 - ID: 2637525490
Texto do Anúncio: Excelente apartamento mobiliado à venda no Residencial São Lourenço bairro Canadá em Cascavel possui...
Banner de cookies não encontrado ou já aceito.
Tempo entre clique e retorno: 7.14 segundos
Rolando um pouco a página...

Processando imóvel 2 - ID: 2734956032
Texto do Anúncio: Excelente apartamento mobiliado à venda no Residencial São Lourenço bairro Canadá em Cascavel possui...
Banner de cookies não encontrado ou já aceito.
Tempo entre clique e retorno: 6.88 segundos
Rolando um pouco a página...

Processando imóvel 3 - ID: 2759542723
Texto do Anúncio: Excelente apartamento mobiliado à venda no Residencial São Lourenço bairro Canadá em Cascavel possui...
Banner de cookies não encontrado ou já aceito.
Tempo entre clique e retorno: 6.51 segundos
Rolando um pouco a página...

Processando imóvel 4 - ID: 2760817014
Texto do Anúncio: Excelente apartamento mobiliado à venda no Residencial São Lourenço bairro Canadá e

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=131.0.6778.109)
Stacktrace:
	GetHandleVerifier [0x005F34A3+25059]
	(No symbol) [0x0057CEA4]
	(No symbol) [0x0045BEC3]
	(No symbol) [0x0043D93B]
	(No symbol) [0x004C800F]
	(No symbol) [0x004DAE49]
	(No symbol) [0x004C1C96]
	(No symbol) [0x00493FAC]
	(No symbol) [0x00494F3D]
	GetHandleVerifier [0x008E5613+3113811]
	GetHandleVerifier [0x008FA2DA+3199002]
	GetHandleVerifier [0x008F2AB2+3168242]
	GetHandleVerifier [0x00693310+680016]
	(No symbol) [0x005857ED]
	(No symbol) [0x00582A98]
	(No symbol) [0x00582C35]
	(No symbol) [0x00575890]
	BaseThreadInitThunk [0x76137BA9+25]
	RtlInitializeExceptionChain [0x76FDC0CB+107]
	RtlClearBits [0x76FDC04F+191]


### OK

In [26]:
from unidecode import unidecode

def scrape_zapimoveis():
    options = uc.ChromeOptions()
    options.headless = False
    driver = uc.Chrome(options=options)
    columns = ["ID", "Códigos", "Área", "Quartos", "Banheiros", "Vagas de Carro", "Preço", "Endereço", "Itens Adicionais", "Texto do Anúncio"]
    df = pd.DataFrame(columns=columns)
    processed_ids = load_progress()  # Carrega IDs processados de arquivo

    try:
        driver.get(start_url)
        accept_cookies(driver)

        listings = driver.find_elements(By.CSS_SELECTOR, "h2[data-cy='rp-cardProperty-location-txt']")
        while listings:
            for i in range(len(listings)):
                try:
                    listings = driver.find_elements(By.CSS_SELECTOR, "h2[data-cy='rp-cardProperty-location-txt']")
                    link = listings[i].find_element(By.XPATH, "./ancestor::a").get_attribute("href")
                    property_id = extract_id_from_url(link)

                    if property_id in processed_ids:
                        print(f"Imóvel já processado: ID {property_id}")
                        continue

                    # Coletar texto do anúncio dentro do contêiner
                    try:
                        container = listings[i].find_element(By.XPATH, "./ancestor::div[contains(@class, 'BaseCard_card__content__pL2Vc')]")
                        description_element = container.find_element(By.CSS_SELECTOR, "p.ListingCard_card__description__slBTG[data-cy='rp-cardProperty-description-txt']")
                        raw_text = description_element.text if description_element else "Não encontrado"
                        ad_text = unidecode(raw_text)  # Remove inconsistências de codificação
                    except Exception as e:
                        print(f"Erro ao coletar texto do anúncio no imóvel {i + 1}: {e}")
                        ad_text = "Não encontrado"

                    print(f"\nProcessando imóvel {i + 1} - ID: {property_id}")
                    print(f"Texto do Anúncio: {ad_text[:100]}...")  # Mostra uma parte do texto para depuração

                    processed_ids.add(property_id)  # Adiciona ID ao conjunto de processados
                    save_progress(processed_ids)  # Salva progresso continuamente

                    start_time = time.time()
                    WebDriverWait(driver, 5).until(EC.element_to_be_clickable(listings[i])).click()
                    WebDriverWait(driver, 10).until(EC.number_of_windows_to_be(2))
                    driver.switch_to.window(driver.window_handles[1])
                    accept_cookies(driver)

                    data = scrape_property_data(driver, property_id, ad_text)
                    df = pd.concat([df, pd.DataFrame([data])], ignore_index=True)

                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])
                    elapsed_time = time.time() - start_time
                    print(f"Tempo entre clique e retorno: {elapsed_time:.2f} segundos")

                    scroll_a_little(driver)
                except Exception as e:
                    print(f"Erro ao processar imóvel: {e}")

            # Tenta avançar para a próxima página
            if not click_next_page(driver):
                break  # Sai do loop caso não haja mais páginas

            # Atualiza a lista de imóveis para a nova página
            listings = driver.find_elements(By.CSS_SELECTOR, "h2[data-cy='rp-cardProperty-location-txt']")

    finally:
        driver.quit()
        df.to_csv("imoveis_zapimoveis.csv", index=False, encoding="utf-8")
        print("Dados salvos no arquivo 'imoveis_zapimoveis.csv'.")

if __name__ == '__main__':
    scrape_zapimoveis()

Cookies aceitos.

Processando imóvel 1 - ID: 2745645149
Texto do Anúncio: Condominio Residencial Golden Endereco: Rua Tuiuti 1239, Cancelli Cascavel-Parana Informacoes do apa...
Banner de cookies não encontrado ou já aceito.
Tempo entre clique e retorno: 5.71 segundos
Rolando um pouco a página...

Processando imóvel 2 - ID: 2728991015
Texto do Anúncio: Lindo Apartamento Central no Edificio Magnifique de alto Padrao. O valor se refere a uma unidade COM...
Banner de cookies não encontrado ou já aceito.
Tempo entre clique e retorno: 6.08 segundos
Rolando um pouco a página...

Processando imóvel 3 - ID: 2649018758
Texto do Anúncio: No Residencial Vinicius voce e sua familia vao descobrir o que e morar bem com conforto e seguranca....
Banner de cookies não encontrado ou já aceito.
Tempo entre clique e retorno: 6.23 segundos
Rolando um pouco a página...

Processando imóvel 4 - ID: 2760817014
Texto do Anúncio: O Residencial Alberto Rodrigues Pompeu, oferece a voce e sua familia uma excelente 

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=131.0.6778.109)
Stacktrace:
	GetHandleVerifier [0x009634A3+25059]
	(No symbol) [0x008ECEA4]
	(No symbol) [0x007CBEC3]
	(No symbol) [0x007AD93B]
	(No symbol) [0x0083800F]
	(No symbol) [0x0084AE49]
	(No symbol) [0x00831C96]
	(No symbol) [0x00803FAC]
	(No symbol) [0x00804F3D]
	GetHandleVerifier [0x00C55613+3113811]
	GetHandleVerifier [0x00C6A2DA+3199002]
	GetHandleVerifier [0x00C62AB2+3168242]
	GetHandleVerifier [0x00A03310+680016]
	(No symbol) [0x008F57ED]
	(No symbol) [0x008F2A98]
	(No symbol) [0x008F2C35]
	(No symbol) [0x008E5890]
	BaseThreadInitThunk [0x76137BA9+25]
	RtlInitializeExceptionChain [0x76FDC0CB+107]
	RtlClearBits [0x76FDC04F+191]


### Adicionando código do anunciante

In [21]:
def scrape_property_data(driver, property_id, ad_text):
    """Coleta os dados de um imóvel na aba atual."""
    data = {"ID": property_id}  # Adicionar ID como chave primária

    # Coletar características principais
    try:
        features = [feature.text for feature in driver.find_elements(By.CSS_SELECTOR, "span[data-cy='ldp-propertyFeatures-txt']")]
        data['Área'] = features[0] if len(features) > 0 else "Não encontrado"
        data['Quartos'] = features[1] if len(features) > 1 else "Não encontrado"
        data['Banheiros'] = features[2] if len(features) > 2 else "Não encontrado"
        data['Vagas de Carro'] = features[3] if len(features) > 3 else "Não encontrado"
    except Exception:
        data['Área'] = data['Quartos'] = data['Banheiros'] = data['Vagas de Carro'] = "Não encontrado"

    # Coletar preço
    try:
        meta_description = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "meta[name='description']"))
        )
        description_content = meta_description.get_attribute("content")
        price_match = re.search(r"R\$ [\d.,]+", description_content)
        data['Preço'] = price_match.group(0) if price_match else "Não encontrado"
    except TimeoutException:
        data['Preço'] = "Não encontrado"

    # Coletar endereço
    try:
        address_element = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "p.address-info-value"))
        )
        data['Endereço'] = address_element.text
    except TimeoutException:
        data['Endereço'] = "Não encontrado"

    # Coletar itens adicionais
    try:
        additional_items = [item.text for item in driver.find_elements(By.CSS_SELECTOR, "span.amenities-item-text[data-cy='ldp-propertyFeatures-txt']")]
        data['Itens Adicionais'] = ", ".join(additional_items) if additional_items else "Não encontrado"
    except Exception:
        data['Itens Adicionais'] = "Não encontrado"

    # Coletar códigos do anunciante e Zap
    try:
        codes_element = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "p[data-cy='ldp-propertyCodes-txt']"))
        )
        data['Códigos'] = codes_element.text
    except TimeoutException:
        data['Códigos'] = "Não encontrado"

    # Usar o texto do anúncio coletado previamente
    data['Texto do Anúncio'] = ad_text

    return data

### Imóveis com anúncios duplicados

In [ ]:
import time
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

In [ ]:
def scroll_until_element(driver, element_selector):
    """Rola lentamente a página até que o elemento especificado apareça e clique nele."""
    print("Rolando a página para encontrar o botão de anúncios duplicados...")
    while True:
        try:
            # Verificar se o botão está visível na página
            button = WebDriverWait(driver, 1).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, element_selector))
            )
            print("Botão de anúncios duplicados encontrado. Clicando...")
            button.click()
            break
        except TimeoutException:
            # Continuar rolando caso o botão não seja encontrado
            driver.execute_script("window.scrollBy(0, 500);")
            time.sleep(1)

In [ ]:
def interact_with_sidebar(driver):
    """Interage com os elementos dentro da aba lateral e coleta informações."""
    print("Interagindo com a aba lateral...")
    collected_data = []
    try:
        # Aguardar até que os elementos de preço estejam disponíveis
        price_elements = WebDriverWait(driver, 5).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "h2.MainValue_advertiser__total__1ornY"))
        )
        for idx, element in enumerate(price_elements):
            print(f"\tClicando no elemento {idx + 1} com preço: {element.text}")
            ActionChains(driver).move_to_element(element).click().perform()
            time.sleep(2)  # Simula interação humana

            # Alterna para a nova guia
            driver.switch_to.window(driver.window_handles[-1])
            print(f"\tNova guia aberta: {driver.current_url}")

            # Extrair ID do imóvel e dados da propriedade
            property_id = extract_id_from_url(driver.current_url)
            property_data = scrape_property_data(driver, property_id)
            collected_data.append(property_data)

            # Fecha a guia atual
            driver.close()
            print("\tGuia fechada.")

            # Retorna para a guia original
            driver.switch_to.window(driver.window_handles[0])
            print("\tRetornando para a aba principal.")
    except TimeoutException:
        print("Elementos de preço não encontrados na aba lateral.")
    finally:
        print("Interação com a aba lateral concluída.")
    return collected_data

In [ ]:
def main():
    # Configurar o driver com undetected_chromedriver
    options = uc.ChromeOptions()
    options.headless = False  # Não ocultar o navegador para debug, pode ser alterado para True
    driver = uc.Chrome(options=options)

    driver.get("https://www.zapimoveis.com.br/venda/apartamentos/pr+cascavel/?pagina=1")
    accept_cookies(driver)
    driver.maximize_window()

    try:
        # Rola a página até encontrar e clicar no botão de anúncios duplicados
        scroll_until_element(driver, "button[data-cy='listing-card-deduplicated-button']")

        # Interage com a aba lateral e coleta os dados
        property_data_list = interact_with_sidebar(driver)

        # Itera e imprime os dados coletados
        print("\nDados coletados:")
        for property_data in property_data_list:
            print(property_data)
    except Exception as e:
        print(f"Erro durante a execução: {e}")
    finally:
        driver.quit()
        print("Script finalizado.")

if __name__ == "__main__":
    main()